# Сбор данных с HH.ru



## Задача

Сбор описаний вакансий с портала hh.ru.

## Референсы

Официальное API. Документация: https://github.com/hhru/api.

Модель данных вакансии: https://github.com/hhru/api/blob/master/docs/vacancies.md


Через API доступны все вакансии, которые не были удалены работодателями. Самые старые доступные вакансии датированы 2008 годом.

В текущей работе мы ограничим временной промежуток до последних трех лет, чтобы не учитывать в исследовании рыночные факторы, такие как инфляция, спрос на специальности и прочее. 

## Формат и модели API

Для получения полного текста API выполняется запрос в эндпойнт /vacancied/{id}. ID вакансии присваиваются хронологически в порядке публикации.

Пример модели данных вакансии представлен ниже.

In [ ]:
! curl -k -H 'User-Agent: api-test-agent' 'https://api.hh.ru/vacancies/100' | python -m json.tool

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2940  100  2940    0     0   2727      0  0:00:01  0:00:01 --:--:--  2727
{
    "id": "100",
    "premium": false,
    "billing_type": {
        "id": "standard",
        "name": "\u0421\u0442\u0430\u043d\u0434\u0430\u0440\u0442"
    },
    "relations": [],
    "name": "\u041f\u0440\u043e\u0434\u0430\u0432\u0435\u0446-\u043a\u043e\u043d\u0441\u0443\u043b\u044c\u0442\u0430\u043d\u0442",
    "insider_interview": null,
    "response_letter_required": false,
    "area": {
        "id": "1",
        "name": "\u041c\u043e\u0441\u043a\u0432\u0430",
        "url": "https://api.hh.ru/areas/1"
    },
    "salary": {
        "from": null,
        "to": 400,
        "currency": "USD",
        "gross": null
    },
    "type": {
        "id": "open",
        "name": "\u041e\u0442\u043a\u0440\u044b\u0442\u0430\u044f"
    },
    "address": n

## Сбор данных

Пользуясь информацией выше, возьмем один из наиболее свежих id вакансий (https://api.hh.ru/vacancies/74118051) и начнем сбор с него, декрементируя. Данные будем предварительно сохранять в текстовые файлы формата json. Выставим слип на случай ограничения количества запросов.

In [22]:
import datetime
import json
import random
import requests
import time

from pathlib import Path
from tqdm.notebook import trange


API_ENDPOINT = "https://api.hh.ru/vacancies/"
WORK_DIR = Path("/home/ephobia/Desktop/HH")
LOG_FILE_PATH = WORK_DIR / "log.txt"
FILE_PATH = WORK_DIR / "data"
START_ID = 74090273 # 74118051
END_ID = 44118051

In [23]:
FILE_PATH.mkdir(parents=True, exist_ok=True)

hh.ru ограничивает количество запросов в открытый API в сутки с одного устройств до нескольких тысяч в сутки, существенно снижает эффективность краулинга. Поскольку итерация по id вакасий не гарантирует, что запрошенная вакансия существует в открытом доступе (она может быть скрыта, находиться в черновиках или удалена), эффективность перебора составлят около 15%.

Чтобы ускорить сбор данных, воспользуемся техникой IP address Spoofing. Были использованы бесплатные прокси, источник: https://free-proxy-list.net/

In [24]:
PROXIES = [
    'http://51.159.115.233:3128',
    '179.96.28.58:80',
    '95.56.254.139:3128',
    '95.154.255.194:8000',
    '116.202.165.119:3121',
    '52.253.83.186:8090',
    '181.143.225.173:3129'
]

In [25]:
def get_proxy():
    proxies = {
       'https': random.choice(PROXIES)
    }
    return proxies

In [26]:
def write_log(idx: str, text: str):
    with open(str(LOG_FILE_PATH), 'a') as fd:
        fd.write(f"{idx} - {datetime.datetime.now()} - {text}\n")
        fd.close()

In [ ]:
for i in trange(START_ID, END_ID, -1):

    try:
        proxies = get_proxy()
        response = requests.get(f"{API_ENDPOINT}{i}", 
                                headers={"User-Agent": "api-test-agent"},
                                proxies=proxies
                               )
        
        if response.status_code == 404:
            continue

        if not response.ok:
            write_log(i, f"{response.status_code} {response.content}")
            break
        
        fp = FILE_PATH / f"{i}.json"
        data = response.json()
        with open(str(fp), 'w') as fd:
            json.dump(data, fd, ensure_ascii=False)

    except Exception as e:
        write_log(i, e)

  0%|          | 0/29972222 [00:00<?, ?it/s]